In [3]:
import pandas as pd

import pickle
from custom_functions import Spotify_function
from fuzzywuzzy import fuzz

In [5]:
missing = pd.read_csv('data/missing_modified_results.csv')
missing.head()

,artists,duration_ms,explicit,id,modified_query,name,popularity
0,"2 Chainz, Gucci Mane, Quavo",222706.0,True,39pS70eeDvyCAF3t8NAlVV,quavo good drank,Good Drank,65.0
1,"21 Savage, Metro Boomin",148640.0,True,2D2w9943rsnJOGCrI4aMQp,21 savage my choppa,My Choppa Hate Niggas,67.0
2,Marillo,191555.0,False,6ItVmJiq8rDwBKq026zSa3,becky g pijama,Como Becky G Sin Pijama,0.0
3,NaN,NaN,NaN,NaN,brett wanna be that song,NaN,NaN
4,NaN,NaN,NaN,NaN,jayz caught their eyes,NaN,NaN


In [13]:
sp = Spotify()

In [14]:
test_results = sp.search(q=missing['modified_query'].values[0])
type(test_results)

dict

In [15]:
def get_best_results(query):
    match_ratio = []
    results = sp.search(q=query)
    best_result_dict = {}
    try:
        for index, result in enumerate(results['tracks']['items']):
            match_ratio.append((index,
                                     fuzz.ratio(query, result['name'])))
        best_result_index = sorted(match_ratio, key=lambda x: x[1], reverse=True)[0][0]
        features = ['id', 'name', 'popularity', 'explicit', 'duration_ms']
        best_result = results['tracks']['items'][best_result_index]
#         best_result_dict = {}
        for feature in features:
            best_result_dict[feature] = best_result[feature]
#         best_result_dict['modified_query'] = query
        best_result_artist = best_result['artists'] # its a list
        artists = [artist['name'] for artist in best_result_artist]
        best_result_dict['artists'] = (', '.join(artists))
    except Exception as e:
        print(e)
        
    best_result_dict['modified_query'] = query
    return best_result_dict


In [16]:
get_best_results(missing['modified_query'].values[0])

{'id': '39pS70eeDvyCAF3t8NAlVV',
 'name': 'Good Drank',
 'popularity': 65,
 'explicit': True,
 'duration_ms': 222706,
 'artists': '2 Chainz, Gucci Mane, Quavo',
 'modified_query': 'quavo good drank'}

In [35]:
results_df  = missing['modified_query'].apply(get_best_results)

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range


In [41]:
results_df = pd.DataFrame.from_records(results_df.values)

In [46]:
# combining with the output

missing_songs = pd.read_csv('data/missing_data_songs.csv')
missing_songs.head()

,artists,duration_ms,explicit,id,name,popularity,modified_query,search_query,acousticness,danceability,...,duration,end_of_fade_in,key_confidence,mode_confidence,num_samples,offset_seconds,start_of_fade_out,tempo_confidence,time_signature_confidence,window_seconds
0,NaN,NaN,NaN,NaN,NaN,NaN,quavo good drank,2 chainz x gucci mane x quavo good drank,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,21 savage my choppa,21 savage my choppa hate n****s,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,becky g pijama,becky g + natti natasha sin pijama,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,brett wanna be that song,brett eldredge wanna be that song,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,jayz caught their eyes,jay-z caught their eyes,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
missing_songs = pd.merge(missing_songs[['modified_query', 'search_query']], results_df, how='left', left_on='modified_query', right_on='modified_query')

In [52]:
missing_songs.head()

,modified_query,search_query,artists,duration_ms,explicit,id,name,popularity
0,quavo good drank,2 chainz x gucci mane x quavo good drank,"2 Chainz, Gucci Mane, Quavo",222706.0,True,39pS70eeDvyCAF3t8NAlVV,Good Drank,65.0
1,21 savage my choppa,21 savage my choppa hate n****s,"21 Savage, Metro Boomin",148640.0,True,2D2w9943rsnJOGCrI4aMQp,My Choppa Hate Niggas,67.0
2,becky g pijama,becky g + natti natasha sin pijama,Marillo,191555.0,False,6ItVmJiq8rDwBKq026zSa3,Como Becky G Sin Pijama,0.0
3,brett wanna be that song,brett eldredge wanna be that song,NaN,NaN,NaN,NaN,NaN,NaN
4,jayz caught their eyes,jay-z caught their eyes,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
missing_songs.dropna(inplace=True)

In [58]:
search_results = pickle.load(open('data/spotify_api_search_results.pkl', 'rb'))
search_results.head()

,artists,duration_ms,explicit,id,name,popularity,search_query
0,"2 Chainz, Travis Scott",255560,True,1nX9KhK3Fff27SnrIor2Yb,4 AM,72,2 chainz 4 am
1,"2 Chainz, Ty Dolla $ign, Trey Songz, Jhene Aiko",210200,True,6H0AwSQ20mo62jGlPGB8S6,It's A Vibe,76,2 chainz it's a vibe
2,"2 Chainz, YG, Offset",234666,True,365wwIjijQdlRJEjUWTidq,PROUD,61,2 chainz proud
3,,,,,,,2 chainz x gucci mane x quavo good drank
4,"2 Chainz, Drake, Quavo",225893,True,5S1IUPueD0xE0vj4zU3nSf,Bigger Than You (feat. Drake & Quavo),75,"2 chainz, drake bigger > you"


In [71]:
# pd.merge(search_results, missing_songs, left_on = 'search_query', right_on='search_query')

search_results = search_results[search_results.popularity!=''].copy(deep=True)
added_missing = pd.concat([search_results, missing_songs]).reset_index()


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


,artists,duration_ms,explicit,id,modified_query,name,popularity,search_query
0,"2 Chainz, Travis Scott",255560,True,1nX9KhK3Fff27SnrIor2Yb,NaN,4 AM,72,2 chainz 4 am
1,"2 Chainz, Ty Dolla $ign, Trey Songz, Jhene Aiko",210200,True,6H0AwSQ20mo62jGlPGB8S6,NaN,It's A Vibe,76,2 chainz it's a vibe
2,"2 Chainz, YG, Offset",234666,True,365wwIjijQdlRJEjUWTidq,NaN,PROUD,61,2 chainz proud
4,"2 Chainz, Drake, Quavo",225893,True,5S1IUPueD0xE0vj4zU3nSf,NaN,Bigger Than You (feat. Drake & Quavo),75,"2 chainz, drake bigger > you"
5,21 Savage,220306,True,2fQrGHiQOvpL9UgPvtYy6G,NaN,Bank Account,83,21 savage bank account
6,21 Savage,234280,True,6g0TAJ1grweRwBMLQyiZFb,NaN,Famous,59,21 savage famous
8,"21 Savage, Metro Boomin",235040,True,4gmmRb6bZJffOOiww1JGTO,NaN,No Heart,73,21 savage no heart
9,"21 Savage, Metro Boomin, Future",258880,True,6fwdbPMwP1zVStm8FybmkO,NaN,X (feat. Future),75,21 savage x
10,"21 Savage, Offset, Metro Boomin, Travis Scott",268933,True,5u6vkDnOyaf8LsteDAj2ub,NaN,Ghostface Killers,75,"21 savage, offset ghostface killers"
11,"21 Savage, Offset, Metro Boomin",202666,True,6HR1z11h0ROUlWP4Kx8Z7G,NaN,Mad Stalkers,65,"21 savage, offset mad stalkers"


In [73]:
added_missing = pd.concat([search_results, missing_songs]).reset_index()
added_missing.drop('index', axis=1, inplace=True)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [74]:
# pickle.dump(added_missing, open('data/search_results.pkl','wb'))

In [44]:
results_df.to_csv('data/missing_modified_results.csv',index=False)

### NTS:

All the missing ones left are JAY Z which makes sense since he has all his songs on tidal instead of his competitor Spoitfy LOL this does make add a little bit of bias but I dont think he's that important 